In [20]:
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



In [21]:
df = pd.read_csv('./data/sonar3.csv', header=None)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.3, shuffle=True) 

In [23]:
# MLP 모델 구성
model = Sequential()
model.add(Dense(24, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(10,  activation='relu'))
model.add(Dense(1,  activation='sigmoid'))

# 모델 컴파일 및 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=16)

Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 0.7145 - accuracy: 0.5172
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 0.7022 - accuracy: 0.5517
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 0.6952 - accuracy: 0.5862
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6881 - accuracy: 0.5172
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 0.6793 - accuracy: 0.5655
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 0.6746 - accuracy: 0.6069
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 0.6709 - accuracy: 0.6138
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 0.6685 - accuracy: 0.6000
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6644 - accuracy: 0.6000
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 0.6573 - accuracy: 0.6276
Epoch 11/

In [24]:
score = model.evaluate(X_test, y_test)
print('Test accuracy : ', score[1])
print('Train accuracy : ', history.history['accuracy'][-1])

2/2 [==============================] - 0s 6ms/step - loss: 0.3907 - accuracy: 0.8254
Test accuracy :  0.8253968358039856
Train accuracy :  0.9724137783050537


In [25]:
# 모델 저장, 재사용

model.save('./data/model/my_model.hdfs')

del  model  # 기존 모델 삭제
model = load_model('./data/model/my_model.hdfs')

INFO:tensorflow:Assets written to: ./data/model/my_model.hdfs\assets


INFO:tensorflow:Assets written to: ./data/model/my_model.hdfs\assets


In [26]:
score = model.evaluate(X_test, y_test)
print('Test accuracy : ', score[1])
print('Train accuracy : ', history.history['accuracy'][-1])

2/2 [==============================] - 0s 4ms/step - loss: 0.3907 - accuracy: 0.8254
Test accuracy :  0.8253968358039856
Train accuracy :  0.9724137783050537


In [29]:
# 작은 데이터를 보완하기 위해 k-교차검증

k = 5   # 데이터를 5겹으로 설정
Kfold = KFold(n_splits=k, shuffle=True)
acc_score = []

def model_func():
    model = Sequential()
    model.add(Dense(24, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10,  activation='relu'))
    model.add(Dense(1,  activation='sigmoid'))
    return model

# k 교차검증
for train_index, test_index in Kfold.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model_func()
    # 모델 컴파일 및 학습
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=0)
    
    score = model.evaluate(X_test, y_test)[1]
    acc_score.append(score)

avg_acc_score = sum(acc_score) / k
    
print('정확도: ', acc_score)
print('정확도 평균: ', avg_acc_score)

2/2 [==============================] - 0s 7ms/step - loss: 4.5911e-07 - accuracy: 1.0000
정확도:  [1.0, 1.0, 1.0, 1.0, 1.0]
정확도 평균:  1.0
